In [4]:
%pip install -U langgraph --quiet

In [9]:
from langgraph.graph import StateGraph, START, END
from langgraph.func import entrypoint, task
from typing import TypedDict
import time
import asyncio

class CandidateState(TypedDict):
    name: str

async def parse_resume_async(state: CandidateState) -> dict:
    print(f"📄 Parsing resume for {state['name']}...")
    await asyncio.sleep(2)
    print(f"   ✅ Resume parsed")
    return {"resume_parsed": True}

async def check_background_async(state: CandidateState) -> dict:
    print(f"🔍 Checking background for {state['name']}...")
    await asyncio.sleep(3)
    print(f"   ✅ Background clear")
    return {"background_check": "Clear"}

async def check_references_async(state: CandidateState) -> dict:
    print(f"📞 Checking references for {state['name']}...")
    await asyncio.sleep(2)
    print(f"   ✅ References positive")
    return {"references": "Positive"}

def create_async_graph():
    workflow = StateGraph(CandidateState)
    workflow.add_node("parse", parse_resume_async)
    workflow.add_node("background", check_background_async)
    workflow.add_node("references", check_references_async)
    workflow.add_edge(START, "parse")
    workflow.add_edge("parse", "background")
    workflow.add_edge("background", "references")
    workflow.add_edge("references", END)
    return workflow.compile()

# Removed @task decorator
async def parse_resume_task_async(name: str) -> dict:
    print(f"📄 Parsing resume for {name}...")
    await asyncio.sleep(2)
    print(f"   ✅ Resume parsed")
    return {"parsed": True}

# Removed @task decorator
async def check_background_task_async(name: str) -> dict:
    print(f"🔍 Checking background for {name}...")
    await asyncio.sleep(3)
    print(f"   ✅ Background clear")
    return {"status": "Clear"}

# Removed @task decorator
async def check_references_task_async(name: str) -> dict:
    print(f"📞 Checking references for {name}...")
    await asyncio.sleep(2)
    print(f"   ✅ References positive")
    return {"status": "Positive"}

# Removed @entrypoint decorator
async def screen_candidate_async(name: str) -> str:
    print(f"\n🎯 Starting screening for {name}...")
    await parse_resume_task_async(name)
    background = await check_background_task_async(name)
    references = await check_references_task_async(name)
    return f"Complete: Background={background['status']}, Refs={references['status']}"

async def main():
    print("="*60)
    print("GRAPH API - ASYNCHRONOUS (Sequential)")
    print("="*60)
    graph_app = create_async_graph()
    graph_state = {"name": "Jane Smith"}
    start_graph = asyncio.get_event_loop().time()
    await graph_app.ainvoke(graph_state)
    total_graph = asyncio.get_event_loop().time() - start_graph
    print(f"\n⏱️  Graph API Total Time: {total_graph:.1f} seconds")
    print(f"   (Still sequential, so 2 + 3 + 2 = 7s)")

    print("\n" + "="*60)
    print("FUNCTIONAL API - ASYNCHRONOUS (Sequential)")
    print("="*60)
    start_func = asyncio.get_event_loop().time()
    await screen_candidate_async("Alice Chen") # Now calling the coroutine directly
    total_func = asyncio.get_event_loop().time() - start_func
    print(f"\n⏱️  Functional API Total Time: {total_func:.1f} seconds")
    print(f"   (Still sequential, so 2 + 3 + 2 = 7s)")

# No need for if __name__ == "__main__": in Colab
# asyncio.run(main()) # Remove this line
await main()

GRAPH API - ASYNCHRONOUS (Sequential)
📄 Parsing resume for Jane Smith...
   ✅ Resume parsed
🔍 Checking background for Jane Smith...
   ✅ Background clear
📞 Checking references for Jane Smith...
   ✅ References positive

⏱️  Graph API Total Time: 7.0 seconds
   (Still sequential, so 2 + 3 + 2 = 7s)

FUNCTIONAL API - ASYNCHRONOUS (Sequential)

🎯 Starting screening for Alice Chen...
📄 Parsing resume for Alice Chen...
   ✅ Resume parsed
🔍 Checking background for Alice Chen...
   ✅ Background clear
📞 Checking references for Alice Chen...
   ✅ References positive

⏱️  Functional API Total Time: 7.0 seconds
   (Still sequential, so 2 + 3 + 2 = 7s)
